In [59]:
# Import Libs
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
import copy
import numpy
import time

from sklearn import datasets, tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from os import system
from IPython.display import Image

%matplotlib inline

In [60]:
# Import the data and name the collums
adultDf = pd.read_csv("adult.data",header=None, index_col=False, 
                      names=['age', 'workclass', 'fnlwgt', "education", "education_num", 
                             "martial-stat", "occupation", "relationship", "race", "sex", 
                             "cap-gains", "cap_loss", "hpw", "native_country", "50k"])

In [61]:
# Give us a first look and check wether it looks ok
adultDf

,age,workclass,fnlwgt,education,education_num,martial-stat,occupation,relationship,race,sex,cap-gains,cap_loss,hpw,native_country,50k
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [62]:
# Replace some Data to make it easier to work with:
# Add an Index:
# adultDf['Index'] = adultDf.index
# >50k -> 1, <=50k ->0
adultDf['50k'] = adultDf['50k'].replace(to_replace=' >50K', value=1)
adultDf['50k'] = adultDf['50k'].replace(to_replace=' <=50K', value=0)

# Education itself is roughly Ordinal, so lets Order it relativly arbitrarily and replace them with Numbers 
# according to the following, where we give the assoiate degrees amd Some collega the same value as they 
# are all rougly equal:
# Preschool, 1st-4th, 5th-6th, 7th-8th, 9th, 10th, 11th, 12th, HS-grad, Assoc-voc, Assoc-acdm, 
# Some-college, Bachelors, Masters, Prof-school, Doctorate,  
adultDf['education'] = adultDf['education'].replace(to_replace=' Preschool', value=0 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 1st-4th', value=1)
adultDf['education'] = adultDf['education'].replace(to_replace=' 5th-6th', value=2 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 7th-8th', value=3)
adultDf['education'] = adultDf['education'].replace(to_replace=' 9th', value=4 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 10th', value=5)
adultDf['education'] = adultDf['education'].replace(to_replace=' 11th', value=6 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 12th', value=7 )
adultDf['education'] = adultDf['education'].replace(to_replace=' HS-grad', value=8 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Assoc-voc', value=9)
adultDf['education'] = adultDf['education'].replace(to_replace=' Assoc-acdm', value=9 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Some-college', value=9 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Bachelors', value=10)
adultDf['education'] = adultDf['education'].replace(to_replace=' Masters', value=11 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Prof-school', value=12 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Doctorate', value=13 )
# Then print the New Dataframe to check:
adultDf

,age,workclass,fnlwgt,education,education_num,martial-stat,occupation,relationship,race,sex,cap-gains,cap_loss,hpw,native_country,50k
0,39,State-gov,77516,10,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,10,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,8,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,6,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,10,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,9,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,8,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,8,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,8,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


## KNN
###### Disclaimer: I (Matthias) wrote a KNN classifier before, so the core functions are copied from my little hobby-learning-project from 2019. Thats why i translate the problem for a classifier instead writing a classifier explicitly for the problem.
### Distance Functions:

##### Distance Euclid

In [77]:
def eukliddist(mat_a, vec_b):
    difference = numpy.subtract(mat_a,vec_b)**2
    difference = difference.sum(axis=1)
    return numpy.sqrt(difference)

In [101]:
def stupideukliddist(mat_a, vec_b):
    difference = [0]*len(mat_a)
    for i in range(len(mat_a)):
        difference[i]=0
        for j in range(len(vec_b)):
            difference[i] = (mat_a[i][j]-vec_b[j])**2 + difference[i]
    return numpy.sqrt(difference)

##### Distance L_1

In [78]:
def L1dist(vec_a, vec_b):
    difference = numpy.subtract(vec_a,vec_b)
    distance = numpy.linalg.norm(difference,axis=1)
    return distance

#### Sorting Tuples

In [79]:
def getKey(item):
    return item[1]

### get the k Nearest Neighbour

In [80]:
def getknn(traindata,trainlabels,newpoint,k):
    # Array of k nearest neighbours
    nearest_neighboors = []
    j = 0
    dist = eukliddist(traindata, newpoint)
    # Loop through Training set
    for i in range(0, len(traindata)):
        entry = (trainlabels[i],dist[1])
        nearest_neighboors.append(entry)
        
    # After we finished the loop repeat the sort and cut
    nearest_neighboors = sorted(nearest_neighboors, key=getKey)
    nearest_neighboors = nearest_neighboors[:k]	
    # Return list of the k nearest neighboors:
    return nearest_neighboors

## The knn Classifier

In [81]:
def knnclassifier(traindata,trainlabels,newpoint,k):
    # Get nearst Neighboors
    neighboorstuple = getknn(traindata,trainlabels,newpoint,k)
    
    # Make them more easy to work with
    neighboorslable = []
    for i in range(0,k):
        neighboorslable.append( str(int(neighboorstuple[i][0])) )

    # Check for majority
    for i in range(0,k):
        count = neighboorslable.count(neighboorslable[i])
        if (count >= (k/2)):
            return int(neighboorstuple[i][0])
    
    # Else return closest Neighboor:
    return int(neighboorstuple[0][0])

### The KNN classifier procedure with timings and results and stuff

In [82]:
def knnprocedure(traindata, trainlabels, testdata, testlabels, k):
    predictedlabels = [0 for x in range(len(testdata))]
    
    # Start Timing Loop
    startT = time.time()
    # Fill Confusion Matrix by iterating at corrct position  len(testlabels))
    for i in range(len(testdata)):
        predictedlabels[i] = knnclassifier(traindata, trainlabels, testdata[i], k)
        
    # Print Runtime
    endT = time.time()
    print("Runtime:" + str(round(endT - startT,3)) )

    # Calculate Classifier Performance
    hits = 0;
    for i in range(len(testlabels)):
        if (testlabels[i]==predictedlabels[i]):
            hits = hits+1

    percentage = hits / len(testlabels)
    print("Accuracy: " + str(round( percentage*100,2) ) + "%" )
    print(' ')

### Prepare the Data, 
#### As the KNN expects numerical data we have to convert some the Categorical Data into Numbers, 
#### We also have to weigh each data axis to standartize them, as mixing binary 0/1 variables with age (0-100+ years) would else be favored by age.

In [92]:
# Copy so not to mess with the original dataframe
adultknn = adultDf.copy()
# Remove fnlwgt(uncorelated) and education (doube data)
adultknn.drop("fnlwgt", axis=1, inplace=True)
adultknn.drop("education", axis=1, inplace=True)
# Add together Capital gains and losses
cap_gains = adultknn["cap-gains"] - adultknn["cap_loss"]
adultknn["cap-gains"] = cap_gains
adultknn.drop("cap_loss", axis=1, inplace=True)
# Repace Sex with 0,1:
adultknn['sex'] = adultknn['sex'].replace(to_replace=' Female', value=1)
adultknn['sex'] = adultknn['sex'].replace(to_replace=' Male', value=0)

# Repace all the complicated relationship status with -0,1 for single, married as that is the important bit:
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Wife', value=1)
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Husband', value=1)
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Unmarried', value=0)
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Other-relative', value=0)
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Not-in-family', value=0)
adultknn['relationship'] = adultknn['relationship'].replace(to_replace=' Own-child', value=0)
# Remove martial-stat as this only doubes reationship status
adultknn.drop("martial-stat", axis=1, inplace=True)
# Remove native Country as turning that into numerical data will either 
# add unwanted ordering (like using the GDP or development Index)
# Or give us the curse of dimensionaity if we add a "isfrom" variable for all countries
adultknn.drop("native_country", axis=1, inplace=True)

# Add 3 binary collums for Workclass, isPrivate, isPublic, isSelfemployed, where a 0 on all means unempoyed or other
adultknn.insert(2, "isPrivate", ([0]*adultknn.shape[0]), True)
adultknn.insert(2, "isPublic", ([0]*adultknn.shape[0]), True)
adultknn.insert(2, "isSelfemployed", ([0]*adultknn.shape[0]), True)
# Now fill them, if we mask multiple times we need to add the earlier 0/1s:
adultknn['isPrivate'] = numpy.where(adultknn['workclass']== ' Private' , 1, 0)
adultknn['isPublic'] = numpy.where(adultknn["workclass"]== ' State-gov' , 1, 0)
adultknn['isPublic'] = numpy.where(adultknn['workclass']== ' Local-gov' , 1, 0)+adultknn['isPublic']
adultknn['isPublic'] = numpy.where(adultknn['workclass']== ' Federal-gov', 1, 0)+adultknn['isPublic']
adultknn["isSelfemployed"] = numpy.where(adultknn['workclass']== ' Self-emp-not-inc', 1, 0)
adultknn["isSelfemployed"] = numpy.where(adultknn['workclass']== ' Self-emp-inc', 1, 0)+adultknn["isSelfemployed"]
# Now we can remove the Workclass
adultknn.drop('workclass', axis=1, inplace=True)


# Add 3 binary collums for Race, isWhite, isBlack, isAsian, 
# Where a 0 on all means AmerIndianEskimo or Other as we only have limited Data on those
adultknn.insert(8, "isWhite", ([0]*adultknn.shape[0]), True)
adultknn.insert(8, "isBlack", ([0]*adultknn.shape[0]), True)
adultknn.insert(8, "isAsian", ([0]*adultknn.shape[0]), True)
adultknn['isWhite'] = numpy.where(adultknn['race']== ' White' , 1, 0)
adultknn['isBlack'] = numpy.where(adultknn["race"]== ' Black' , 1, 0)
adultknn['isAsian'] = numpy.where(adultknn['race']== ' Asian-Pac-Islander' , 1, 0)
adultknn.drop('race', axis=1, inplace=True)


# Change Occupation into a Variable Field which is:
# -1 for low paying fields, 0 for normal fields and 1 for high paying fields
adultknn.insert(5, "occField", ([0]*adultknn.shape[0]), True)
# 2 High paying fields >40% over 50k
adultknn["occField"] = numpy.where(adultknn['occupation']== ' Exec-managerial' , 1, 0)+adultknn["occField"]
adultknn["occField"] = numpy.where(adultknn['occupation']== ' Prof-specialty' , 1, 0)+adultknn["occField"]
# 3 low paying fields, <10% over 50k
adultknn["occField"] = numpy.where(adultknn['occupation']== ' Handlers-cleaners' , -1, 0)+adultknn["occField"]
adultknn["occField"] = numpy.where(adultknn['occupation']== ' Other-service' , -1, 0)+adultknn["occField"]
adultknn["occField"] = numpy.where(adultknn['occupation']== ' Priv-house-serv' , -1, 0)+adultknn["occField"]
adultknn.drop('occupation', axis=1, inplace=True)

# STANDARTIZE NOW:
adultknn['education_num'] = adultknn['education_num'].apply(lambda x: x/13*1)
column = adultknn["age"]
max_age = column.max()
adultknn['age'] = adultknn['age'].apply(lambda x: x/max_age*1)
column = adultknn["cap-gains"]
max_capgains = column.max()
adultknn['cap-gains'] = adultknn['cap-gains'].apply(lambda x: x/max_capgains*1)
column = adultknn["hpw"]
max_hpw = column.max()
adultknn['hpw'] = adultknn['hpw'].apply(lambda x: x/max_hpw*1)
# Lets look at the Table Now
print(adultknn)

# Cut some Items out to be our Testing set and the rest as training set
# I choose to ony do 5% (roughly 1500 samples) so i will not wait for hours on my old shitty laptop!
adultknn_test = adultknn.sample(frac=0.05)
adultknn.drop(adultknn_test.index)
# Put the Labels into a different vector:
adultknnlabels = adultknn["50k"].to_numpy()
adultknn.drop('50k', axis=1, inplace=True)
adultknndata = adultknn.to_numpy()

adultknntestlabels = adultknn_test["50k"].to_numpy()
adultknn_test.drop('50k', axis=1, inplace=True)
adultknn_testdata = adultknn_test.to_numpy()
# Put the Matrix without the Labels into a Matrix
print(adultknndata)
print(adultknnlabels)

            age  isSelfemployed  isPublic  isPrivate  education_num  occField  \
0      0.433333               0         1          0       1.000000         0   
1      0.555556               1         0          0       1.000000         1   
2      0.422222               0         0          1       0.692308        -1   
3      0.588889               0         0          1       0.538462        -1   
4      0.311111               0         0          1       1.000000         1   
...         ...             ...       ...        ...            ...       ...   
32556  0.300000               0         0          1       0.923077         0   
32557  0.444444               0         0          1       0.692308         0   
32558  0.644444               0         0          1       0.692308         0   
32559  0.244444               0         0          1       0.692308         0   
32560  0.577778               1         0          0       0.692308         1   

       relationship  isAsia

## After all that we Data preperation we can now finaly work with the KNN:

### Lets run the KNN and check how the k parameter behaves and how much time it needs
#### Our Test size is a random sample (not in the trainingset of course) of Size 5%, as more is to big for my old venerable Thinkpad

In [95]:
print("Size of Trainingset: " + str(len(adultknnlabels)) + " and Size of Testset: " + str(len(adultknntestlabels)) )
print("For k=1")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 1)
print("For k=3")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 3)
print("For k=5")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 5)
print("For k=9")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 9)
print("For k=13")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 13)

Size of Trainingset: 32561 and Size of Testset: 1628
For k=1
Runtime:88.727
Accuracy: 94.04%
 
For k=3
Runtime:81.03
Accuracy: 89.8%
 
For k=5
Runtime:80.504
Accuracy: 87.71%
 
For k=9
Runtime:128.823
Accuracy: 86.43%
 
For k=13
Runtime:82.123
Accuracy: 85.32%
 


#### We have a predictive power in the low 90% for k=1, and sinking slightly until reaching 85%. And the time it takes is always roughly 85s every time
#### We also want to note those Numbers have not stabalized, and different random Test and Training samples give different accuracys even so low as only 78%
## Now lets compare that to sklearns KNN:

In [97]:
print("KNN for k=1")
startT = time.time()
sklearnknnclass = KNeighborsClassifier(n_neighbors=1)
sklearnknnclass.fit(adultknndata, adultknnlabels)
skscore = sklearnknnclass.score(adultknn_testdata, adultknntestlabels)
endT = time.time()
print("Runtime:" + str(round(endT - startT,3)) )
print("Accuracy: " + str(round(skscore*100,2)) + "%")
print(' ')

print("KNN for k=3")
startT = time.time()
sklearnknnclass = KNeighborsClassifier(n_neighbors=3)
sklearnknnclass.fit(adultknndata, adultknnlabels)
skscore = sklearnknnclass.score(adultknn_testdata, adultknntestlabels)
endT = time.time()
print("Runtime:" + str(round(endT - startT,3)) )
print("Accuracy: " + str(round(skscore*100,2)) + "%")
print(' ')
print("KNN for k=5")
startT = time.time()
sklearnknnclass = KNeighborsClassifier(n_neighbors=5)
sklearnknnclass.fit(adultknndata, adultknnlabels)
skscore = sklearnknnclass.score(adultknn_testdata, adultknntestlabels)
endT = time.time()
print("Runtime:" + str(round(endT - startT,3)) )
print("Accuracy: " + str(round(skscore*100,2)) + "%")
print(' ')
print("KNN for k=9")
startT = time.time()
sklearnknnclass = KNeighborsClassifier(n_neighbors=9)
sklearnknnclass.fit(adultknndata, adultknnlabels)
skscore = sklearnknnclass.score(adultknn_testdata, adultknntestlabels)
endT = time.time()
print("Runtime:" + str(round(endT - startT,3)) )
print("Accuracy: " + str(round(skscore*100,2)) + "%")
print(' ')
print("KNN for k=13")
startT = time.time()
sklearnknnclass = KNeighborsClassifier(n_neighbors=13)
sklearnknnclass.fit(adultknndata, adultknnlabels)
skscore = sklearnknnclass.score(adultknn_testdata, adultknntestlabels)
endT = time.time()
print("Runtime:" + str(round(endT - startT,3)) )
print("Accuracy: " + str(round(skscore*100,2)) + "%")
print(' ')

KNN for k=1
Runtime:0.608
Accuracy: 94.04%
 
KNN for k=3
Runtime:0.63
Accuracy: 89.68%
 
KNN for k=5
Runtime:0.676
Accuracy: 87.84%
 
KNN for k=9
Runtime:0.712
Accuracy: 86.36%
 
KNN for k=13
Runtime:0.761
Accuracy: 85.57%
 


### Good News: We perform on paar with the sklearn-KNN classifier.

### Bad News: As we see, the libery is over 100x faster with roughly 0.6s. And i already used the vector multiplication, so there is no easy speed up to find. You just cand beat optimized dark magic bitflipping written in pure C!

### Now of course we could make our KNN slower by replacing the vector euclid distance with nested for-loop calculating the euclidian distance, lol. So of course we gonne try that later.
### For shits and giggles, lets first try the L_1 distance:

In [98]:
def getknn(traindata,trainlabels,newpoint,k):
    # Array of k nearest neighbours
    nearest_neighboors = []
    j = 0
    dist = L1dist(traindata, newpoint)
    # Loop through Training set
    for i in range(0, len(traindata)):
        entry = (trainlabels[i],dist[i])
        nearest_neighboors.append(entry)
        
    # After we finished the loop repeat the sort and cut
    nearest_neighboors = sorted(nearest_neighboors, key=getKey)
    nearest_neighboors = nearest_neighboors[:k]	
    # Return list of the k nearest neighboors:
    return nearest_neighboors

print("Size of Trainingset: " + str(len(adultknnlabels)) + " and Size of Testset: " + str(len(adultknntestlabels)) )
print("For k=1")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 1)
print("For k=3")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 3)
print("For k=5")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 5)
print("For k=9")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 9)
print("For k=13")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 13)

Size of Trainingset: 32561 and Size of Testset: 1628
For k=1
Runtime:68.76
Accuracy: 94.04%
 
For k=3
Runtime:82.337
Accuracy: 89.8%
 
For k=5
Runtime:97.434
Accuracy: 87.71%
 
For k=9
Runtime:89.416
Accuracy: 86.43%
 
For k=13
Runtime:95.492
Accuracy: 85.32%
 


### Same accuracy, runntime a bit more inconsitent, does not help against the liberies.

### Fine, lets run the stupid version of Euclidian distance, but only twice because we dont want to torture my poor old thinkpad to much:

In [103]:
def getknn(traindata,trainlabels,newpoint,k):
    # Array of k nearest neighbours
    nearest_neighboors = []
    j = 0
    dist = stupideukliddist(traindata, newpoint)
    # Loop through Training set
    for i in range(0, len(traindata)):
        entry = (trainlabels[i],dist[i])
        nearest_neighboors.append(entry)
        
    # After we finished the loop repeat the sort and cut
    nearest_neighboors = sorted(nearest_neighboors, key=getKey)
    nearest_neighboors = nearest_neighboors[:k]	
    # Return list of the k nearest neighboors:
    return nearest_neighboors

print("Size of Trainingset: " + str(len(adultknnlabels)) + " and Size of Testset: " + str(len(adultknntestlabels)) )
print("For k=3")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 3)
print("For k=9")
knnprocedure(adultknndata, adultknnlabels, adultknn_testdata, adultknntestlabels, 9)

Size of Trainingset: 32561 and Size of Testset: 1628
For k=3
Runtime:612.364
Accuracy: 89.8%
 
For k=9
Runtime:835.582
Accuracy: 86.43%
 


### And yep, as we see, doing it the stupid way with nested loops takes another 10x the time of the vector version or 1000x the time of the libery. So dont do that ever.